Overview
=========

**Principles:**

1. Get it working.
2. Squabble Over Performance Afterards

**Process:**

## Download Source
- WP 
    - Export file?
    - WP API?
- `[Sq]Space` API?
- Blogger
    - 
- Scrape any WP site?
    - scrape posts?
    - theme chameleon (theme copier)
    
maybe this is an open source tool to get onto their own "fs"

## Normalize Source

- Use pandoc to covert to md file?
- Open Source the Archive Tool due to GPL issues.

## Upload to Dynamo
- Easy Options:
    - Compress & Upload Blob?
    - Pick root folder with all source files contained

### Upload Normalized Source
### (Walk FS & Upload MD Files)
- Walk the OS => get list of abs path files
- Upload each file to s3 => s3://location
- Parse Each MD file for metadata and upload the {metadata & s3://loc to dynamoDB}

## Walk DynamoDB 

- Walk DynamoDB Posts
    - ToDo : Config a Notion of What First
- Drop each MD file into a page builder
- Page Builder injects the \_config, site, page data into a theme
    - multiprocess workers 
        - gets a result 
        - fetch any needed remote assets
        - render a thematic page
        - Place processed html into target s3 bucket
        - each worker returns the processed HTML, post:MD, post:HTML, and `___`?
- Proces sitewide files
    - sitemap
        - look at metamaps
            - post map
                - chunked into blocks
            - tag map
            - category map

    - search blob
        - Create a chunked resource convention
               
           Loose scan over entries(pages/posts/etc)
           estimate/determine number of chunks
           name/number all the chunks
           issue working data to working chunk
               issue context of all chunk IDs
               issue context of working next + prev
               
               search.json :: =>
               {
                   `_meta`:{
                       `epochofBuild`: "12123512351235", // time is a shared attr by all search file chunks
                       "@ref=prev"  :  null,
                       "@ref=next"  : "http://<domain>/<path>/search2.json",
                       "@ref=all"   :["http://<domain>/<path>/search2.json",
                                      "http://<domain>/<path>/search3.json"
                                      "http://<domain>/<path>/search4.json"
                                      "http://<domain>/<path>/search5.json"] // ref=all is a shared attr by all
                   },
                   posts:\[
                       {
                           `_id`:"1234",
                           "localKeys1":"LocalValue1",
                           "localKeys2":{"LocalSuKey1":True}
                           "cotent":{"md"  : `<md:content>`,
                                     "html":`<html:content>`}
                           "plugins":{
                               <pluginName1>:{_meta:{},value:<value1>}
                               <pluginName2>:{_meta:{},value:<value2>}                               
                           }
                            <all keys>:<all values needed to describe a post>
                        },
                       {
                           `_id`:"1235",                           
                           "localKeys1":"LocalValue1",
                           "localKeys2":{"LocalSuKey1":True}
                           "cotent":{"md"  : `<md:content>`,
                                 "html":`<html:content>`}
                       },
                       {
                           `_id`:"1236",                           
                           "localKeys1":"LocalValue1",
                           "localKeys2":{"LocalSuKey1":True}
                           "cotent":{"md"  : `<md:content>`,
                                 "html":`<html:content>`}
                       }
                   ]
               }
               
    - etc
    - ToDo: How to do this without pulling everything into memory

## SDK Start

    mg = Mirage(dydbr)
    
    mg.SavePost(file="abs/path/toFile.md", raw=False)
    mg.SavePost([id], metadata="tags", value="essay", raw=False)

    mg.GetDrafts(ids=[id], raw=False)
    mg.GetDrafts(by="month" value="2017-01", raw=False)
    mg.GetDrafts(by="year" value="2017", raw=False)

    mg.GetDrafts(ids=[id], raw=False)
    mg.GetPosts(by="month" value="2017-01", raw=False)
    mg.GetPosts(by="year" value="2017", raw=False)
    mg.GetPosts(by="title", raw=False)
    mg.GetPosts(by="loc", raw=False)
    mg.GetPosts(by="author", raw=False)
    mg.GetPosts(by=[author,month], value=["Eric","2017-01"], raw=False)

    mg.BulkUpdateTags(ids=[id], raw=False)
    mg.BulkUpdateCategory(ids=[id], raw=False)
    mg.BulkUpload(dir="abs/path/to/dir", raw=False)


Elastic Pagination
===================

When all pages are full, the addition of one post causes an unbounded domino effect propagating changes to the pagination view all the way to the last set - of any length. This happens with every new change. When a fixed number of pages per "paginated index view" are permitted, then all sets, except the last, deal with a FIFO queue bump, sending the oldest page down stream in favor of the new page in. The result is recalculation of every set in the chain of arbitrary length. 

If each page were allowed to have a range of acceptable values, (eg: 8 through 12 inclusive) the addition of a new page would not have to ripple a domino change all the way down the chain.

Permitting heterogeneous page sizes stops the required processing from propagating indefinitely. 

Example: Create a Pagination Policy:
	8 <= items in page <= 12

*Initial Condition:*

- front Page:
	- 10 items
- Second Page:
	- 10 items:
- Third Page:  
	- 8 items
- Fourth Page:  
	- 10 items

THEN (+1 post)

*2:State* (no pagination spill)

- front Page:
	- **11 items**
- Second Page:
	- 10 items:
- Third Page:  
	- 8 items
- Fourth Page:  
	- 10 items

THEN (+1 post)

*3:State (no page spill)*

- front Page:
	- **12 items**
- Second Page:
	- 10 items:
- Third Page:  
	- 8 items
- Fourth Page:  
	- 10 items

THEN (+1 post)

*4:State (Spill Across 4 pages)*

- front Page:
	- **8 items** (dropped to 8 for cushion - sends 5 down)
- Second Page:
	- **12 items**: (catch 2 - send 3 down)
- Third Page:  
	- **11 items** (catch 3)
- Fourth Page:  
	- 10 items

THEN (+1 post)

*5:State (no spill)*

- front Page:
	- **9 items**
- Second Page:
	- 12 items:
- Third Page:  
	- 11 items
- Fourth Page:  
	- 10 items

**Add a Background Process** - "cushion balancing" lowers the page per index on the front pages to accommodate new additions, it also makes "cushion" pages through out the index pagination chain. So that the processing propagation hits cushions and stay for short lengths.

Creating "cushion/gap pages" allow processing the "fast path" to stay "fast". Ensuring *cushion pages* deters the dominoes from falling all the way down the chain. Aka: when you +1ing a post, the pagination chain gets bumped, and we do not intend the chain to get wholly processed.

**Background Processing**

Evens out the changes needed for each

Updating A Site
======================

1. Look at current site state(0)
2. Look at the needs for new state(1)
3. Perform a diff


Diff is actually an optimization
- Naieve approach is delete the whole thing and make generate state1 from scratch.
- Issue changes to achieve new state


Create/Edit the Asset and Update/Save the Site-State

0. Post/Page Rendering (permalink scope)
1. Front Index Page
2. Pagination Index Pages - use stretching pages
3. Search Assets - can be append only
4. Tags
5. Categories
6. Series

Other Site Type with related types:

- Movies
- Actors
- Directors
- Studios
- Genre
- Year
- Awards

Add front material for a pre-created Movie Entry
Add front material for a pre-created Actor Entry

Hugo Example:

Taxonomy/Term/Value

- /movies/\_index
    - featured movies
    - featured images
- /movies/:year/:slug || :title
    - add bio
    - add images
    - add video

- /actors/\_index
    - featured Actors
    - featured Actor News?
- /actors/:name
    - (precreated via movie entry)
        - comes with a list of related movies
    - add bio
    - add images
    - add video

- /directors/\_index
    - featured Directors
    - featured Director?
- /directors/:name
    - (precreated via movie entry)
        - comes with a list of related movies
    - add bio
    - add images
    - add video


- /studios/\_index
    - featured Studio
    - featured Studio
- /studios/:name
    - (precreated via movie entry)
        - comes with a list of related movies
    - add bio
    - add images
    - add video


- /genres/\_index
    -  Genre News
- /genres/:name
    - (precreated via movie entry)
        - comes with a list of related movies
    - add bio
    - add images
    - add video

- /years/\_index
    - Best of List
    - Box Office News
    - Upcoming
- /years/:name
    - (precreated via movie entry)
        - comes with a list of related movies
    - add bio
    - add images
    - add video

- /awards/\_index
    - Best of List
    - Box Office News
    - Upcoming
- /awards/:name


OR - (etc..)
- /person/actor
- /person/director
- /person/cinematographer



Each of the data buckets appended to the "Movie" the nexus of all the 
